In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import argparse

def split_tool(url):
    """
    將movie id摘取出來，並加上base url="https://movies.yahoo.com.tw/movieinfo_main.html/id="，
    最後回傳該電影頁面的完整url
    
    """
    base="https://movies.yahoo.com.tw/movieinfo_main.html/id="
    result = base+url.split('-')[-1]
    return(result)

def get_movieid(home_page):
    """
    Yahoo奇摩電影網頁為get，不同的電影，對應不同的id,
    此處抓的是"上映中"頁籤中所有的電影
    https://movies.yahoo.com.tw/movie_intheaters.html
    
    """
    soup = BeautifulSoup(get_web_pages(home_page),'html.parser')
    a=map(split_tool,re.findall("href=\"(.*?)\">",str(soup.select('div[class="release_foto"]'))))
    urls = list(a)
    return urls

def surfing_all_page(base_url):
    """
    一頁10筆電影,迴圈直至抓完所有的頁數，
    共"電影數/10" 頁
    
    """
    url_list =[]
    soup = BeautifulSoup(get_web_pages(base_url),'html.parser')
    num_movies = int(re.search("共(\d+)筆",soup.find('div','release_time _c').p.text).group(1))
    num_pages = np.ceil(num_movies/10).astype(int)
    each_page=base_url+'?page={}'
    for i in range(1,num_pages+1):
        url_list +=get_movieid(each_page.format(i))
    
    return url_list

def get_web_pages(url):
    """
    傳送request，並設置"已滿18歲"
    
    """
    req = requests.get(url,cookies={'over18':'1'})
    if req.status_code!=200:
        print('Invalid web:',url)
        return None
    else:
        return req.text

def get_movies(url):
    """
    爬取電影名稱(中)、電影名稱(英)、期待度、滿意度、
    類型、演員、導演、官方連結、劇情介紹，並儲存為dictionary
    
    """
    soup = BeautifulSoup(get_web_pages(url),'html.parser')
    dicts={}
    dicts['電影名稱(中)'] = soup.h1.text
    dicts['電影名稱(英)'] = soup.h3.text
    dicts['期待度'] = soup.find('div','circle_percent')['data-percent']+'%'
    dicts["滿意度"] = soup.find('div',"score_num count").text.strip()

    content = str(soup.find('div','movie_intro_info_r'))
    dicts["類型"] = '/'.join(re.findall("電影介紹_類型icon\\'\,\\'(.*?)\\'",content))




    for i in soup.find('div',"movie_intro_info_r").find_all('span'):
        try:
            title =i.text.strip().split("：")[0]
            title = re.sub("\u3000","",title)
            dicts[title]=i.text.strip().split("：")[1]
        except:
            pass

    dicts["演員"] = re.sub("\s+","",'/'.join(re.findall("電影介紹_演員資訊\\'\,\\'(.*?)\\'",content)))
    dicts["導演"] = re.sub("\s+","",soup.find('div',"movie_intro_list").text.strip())
    dicts["官方連結"] = '/'.join(re.findall("電影介紹_官網1\\'\,\\'(.*?)\\'",content))

    dicts["劇情介紹"] = soup.find('div','gray_infobox_inner').span.text.strip()
    
    return dicts

def create_movie_df(base_url):
    """
    將get_movie獲得的dictionary下載為dataframe
    
    """
    urls = surfing_all_page(base_url)
    movies=[]
    for url in urls:
        movies.append(get_movies(url))
    result = pd.DataFrame(movies,columns=movies[0].keys())
    return result

In [4]:
base_url="https://movies.yahoo.com.tw/movie_intheaters.html"

create_movie_df(base_url)

,電影名稱(中),電影名稱(英),期待度,滿意度,類型,上映日期,片長,發行公司,IMDb分數,導演,演員,官方連結,劇情介紹
0,波希米亞狂想曲,Bohemian Rhapsody,95%,4.7,音樂/歌舞/歷史/傳記,2018-11-02,02時15分,福斯影片,8.4,戴克斯特佛萊契(DexterFletcher),雷米馬利克/露西波頓,https://www.facebook.com/foxmovies.tw,★搖滾不死！皇后合唱團傳記電影首登大銀幕 神曲百分百催淚再詮釋★《波希米亞狂想曲》傳奇主唱佛...
1,護墊俠,Pad Man,74%,4.4,歷史/傳記/劇情/喜劇,2018-11-02,02時17分,車庫,8.1,R.巴勒庫許南(R.Balki),,https://www.facebook.com/garageplay.tw,★ 改編自不可思議的真人真事，又一印度神作
2,那些年，我們一起追的女孩(日),You're The Apple Of My Eye,75%,4,劇情,2018-11-02,01時55分,天馬行空,NaN,長谷川康夫(YasuoHasegawa),長谷川康夫/山田裕貴/齋藤飛鳥,http://anokoro-kimio.jp/,九把刀原作《那些年，我們一起追的女孩》翻拍日本版再度搬上大銀幕!
3,生命中的美好意外,Life Itself,83%,4,劇情,2018-11-02,01時57分,傳影互動,5.8,丹福吉曼(DanFogelman),丹福吉曼/奧斯卡伊薩克/奧莉維亞魏爾德/奧利薇亞庫克/安妮特班寧,http://www.fb.com/ifilm.tw,沒有比生命本身 更偉大的故事
4,嚇地獄,Hell Fest,78%,3.4,恐怖/懸疑/驚悚,2018-11-02,01時29分,CatchPlay,5.8,葛瑞格里普洛金(GregoryPlotkin),,http://www.facebook.com/catchplay,★ 《陰屍路》金牌製作班底 最新嗜血恐懼新作
5,親愛的卵男日記,Bao Bao,76%,3.6,愛情/劇情,2018-11-02,未提供,星泰娛樂,NaN,謝光誠,,https://www.facebook.com/SHINETIME2017/,★ 繼李安導演《囍宴》徐立功監製《滿月酒》後再創同志話題力作
6,誰先愛上他的,Dear EX,97%,4.7,劇情,2018-11-02,01時40分,華納兄弟,7.5,徐譽庭、許智彥,,,一個十四歲的臭屁男孩，因為父親病逝後留下的保險受益人被竄改，而成為這場爭奪戰的客觀視角，故事...
7,我的嗝嗝老師,Hichki,96%,4.5,劇情/喜劇,2018-11-02,01時57分,捷傑電影、水元素,7.6,SiddharthMalhotra,,,立志當老師的奈娜，準備在教育界大展雌威，無奈她有個無法隱藏 的「小」缺陷，嗯，或許也沒那麼小。
8,親密陌生人,Intimate Strangers,83%,3.7,劇情,2018-11-02,01時55分,采昌國際多媒體,6.2,李在奎(LEEJae-kyoo),李在奎/柳海真/趙震雄/廉晶雅/李瑞鎮/金志秀/松昰昀/尹卿鎬,,★一場手機遊戲，你願意為愛情付出多少代價？
9,夜驚,Mara,85%,3.3,恐怖/懸疑/驚悚,2018-11-02,01時38分,牽猴子整合行銷股份有限公司,5.1,克里夫湯奇(CliveTonge),歐嘉柯瑞蘭蔻,https://www.facebook.com/monkeymovies/,★ 《陰兒房》《鬼入鏡》團隊又一駭人懼作
